# Clasificación de nodos

En esta práctica se realiza la clasificación de nodos para distintos datasets empleando varios modelos. Se evalúa su rendimiento mediante el error de entrenamiento y se comparan con el fin de establecer el óptimo en cada caso.

## Datasets



Los datasest que se tienen se pueden diferenciar según su homofilia y heterofilia. Estos conceptos en el ámbito de los grafos se refieren a las propiedades de conexión entre los nodos basadas en sus características o etiquetas.

En un grafo homofílico, los nodos tienden a conectarse con otros nodos que son similares a ellos. Esta similitud puede basarse en varias características, como etiquetas, atributos o cualquier otra propiedad relevante. En otras palabras, en un grafo homofílico, los nodos con características similares están más probablemente conectados entre sí.

En un grafo heterofílico, los nodos tienden a conectarse con otros nodos que son diferentes a ellos. En este caso, la conexión entre nodos ocurre entre aquellos con características disímiles. Esto es menos común que la homofilia en muchos contextos, pero puede ser relevante en ciertas aplicaciones.

La distinción entre grafos homofílicos y heterofílicos es importante en el contexto de machine learning sobre grafos, especialmente cuando se usan modelos de redes neuronales de grafos (GNN). Los modelos GNN tradicionales, como GCN (Graph Convolutional Network), tienden a funcionar mejor en grafos homofílicos, donde la agregación de características de nodos vecinos similares refuerza las señales útiles para la tarea de aprendizaje. Sin embargo, en grafos heterofílicos, la agregación de características de nodos vecinos disímiles puede introducir ruido, lo que hace que estos modelos sean menos efectivos.



La **homofilia** de un grafo se puede definir como la proporción de aristas que conectan nodos con la misma etiqueta.

Sea $G = (V, E)$ un grafo donde:
- $V$ es el conjunto de nodos.
- $E$ es el conjunto de aristas.

Cada nodo $v \in V$  tiene una etiqueta asociada $y(v)$.

La homofilia se puede calcular de la siguiente manera:

$$
\text{Homofilia} = \frac{|\{ (u, v) \in E \mid y(u) = y(v) \}|}{|E|}
$$

donde:
- $|\{ (u, v) \in E \mid y(u) = y(v) \}|$ es el número de aristas que conectan nodos con la misma etiqueta.
- $|E|$ es el número total de aristas en el grafo.

Se calcula el grado de homofilia para cada uno de los conjuntos de datos, cuanto más se acerque su valor a 1 más homofílico será el grafo. Los resultados son los siguientes:

**Grafos homofílicos**: Cora (0.8100), Citeseer (0.7355)

**Grafos heterofílicos**: Actor (0.2188), Chameleon (0.2350), Texas (0.1077), Wisconsin (0.1961), Squirrel (0.2239), Cornell (0.1309)



Nota: Los datasets de Chameleon y Squirrel son grafos DGL, y a la hora de tratar con ellos y acceder a sus caracteríticas es necesario utilizar otras funciones.

## Modelos

Los modelos que se van a utilizar en esta práctica son los siguientes:

#### MLP

Un Multilayer Perceptron (MLP) es un tipo de red neuronal artificial que consiste en múltiples capas de nodos (neuronas). En un MLP típico, cada nodo en una capa está conectado a cada nodo en la siguiente capa, formando una red totalmente conectada.

  $ H^{(l+1)} = \sigma(H^{(l)} W^{(l)} + b^{(l)})$

- $ H^{(l)} $: Características de entrada en la capa $ l $.
- $ W^{(l)} $: Pesos de la capa $ l $.
- $ b^{(l)} $: Sesgo de la capa $ l $.
- $ \sigma $: Función de activación.

En general, un MLP puro puede no aprovechar completamente la estructura del grafo, ya que trata a cada nodo de forma independiente sin considerar las conexiones entre nodos. Por lo tanto, en grafos homofílicos, un MLP puro generalmente funciona peor que los modelos basados en GNN (Graph Neural Networks), como GCN, que pueden aprovechar la homofilia para mejorar las predicciones. En algunos casos, un MLP puede funcionar mejor que un modelo GNN en grafos heterofílicos, especialmente si la agregación de nodos vecinos introduce más ruido que señal útil.

#### GCN

  $ H^{(l+1)} = \sigma(D^{-\frac{1}{2}} A D^{-\frac{1}{2}} H^{(l)} W^{(l)}) $

- $H^{(l)}$: Características de los nodos en la capa $l$ .
- $ A $: Matriz de adyacencia del grafo.
- $ D $: Matriz diagonal de grados de los nodos.
- $ W^{(l)} $: Pesos de la capa $ l $.
- $ \sigma $: Función de activación (e.g., ReLU).

Funciona bien con grafos homofílicos, donde los nodos tienen conexiones densas dentro de comunidades o grupos, ya que GCN opera mediante la propagación de información entre los nodos vecinos.


#### SAGE

Este modelo realiza el muestreo de vecinos y agregación de información de los nodos vecinos para aprender representaciones de nodos.

  $ h_v^{(l+1)} = \sigma \left( W^{(l)} \cdot \text{AGGREGATE}^{(l)} \left( \{ h_u^{(l)}, \forall u \in \mathcal{N}(v) \cup \{ v \} \} \right) \right) $

- $ h_v^{(l)} $: Características del nodo $ v $ en la capa $ l $.
- $ \mathcal{N}(v) $: Conjunto de nodos vecinos de $ v $.
- $\text{AGGREGATE}$: Función de agregación (e.g., media, suma, máximo).
- $ W^{(l)} $: Pesos de la capa $ l $.
- $ \sigma $: Función de activación.

Es efectivo para grafos homofílicos y heterofílicos, donde los nodos pueden tener conexiones con diferentes tipos de nodos y la estructura de la red es más diversa. GraphSAGE puede capturar la información local de los vecinos de los nodos de manera efectiva.

#### Cheb

Utiliza polinomios de Chebyshev para aproximaciones de convoluciones en grafos.

  $ H^{(l+1)} = \sigma \left( \sum_{k=0}^{K} T_k(\tilde{L}) H^{(l)} W_k^{(l)} \right) $

- $ T_k(\tilde{L}) $: Polinomio de Chebyshev de orden $ k $ evaluado en el operador de Laplace normalizado $ \tilde{L} $.
- $ \tilde{L} $: Operador de Laplace normalizado.
- $ H^{(l)} $: Características de los nodos en la capa $ l $.
- $ W_k^{(l)} $: Pesos asociados con el polinomio de Chebyshev de orden $ k $ en la capa $ l $.
- $ \sigma $: Función de activación.

Puede ser adecuado para grafos homofílicos donde la estructura local de los nodos es importante. Al aproximar convoluciones en el espectro de Chebyshev, puede capturar eficazmente la información de los vecinos de los nodos.



#### GAT

Utiliza mecanismos de atención para permitir que cada nodo pese la información de sus vecinos de manera diferente durante la agregación de información.


  $ h_i^{(l+1)} = \sigma \left( \sum_{j \in \mathcal{N}(i) \cup \{i\}} \alpha_{ij}^{(l)} W^{(l)} h_j^{(l)} \right) $

$ \alpha_{ij}^{(l)} = \frac{\exp(\text{LeakyReLU}(a^{\top} [W^{(l)} h_i^{(l)} \| W^{(l)} h_j^{(l)}]))}{\sum_{k \in \mathcal{N}(i) \cup \{i\}} \exp(\text{LeakyReLU}(a^{\top} [W^{(l)} h_i^{(l)} \| W^{(l)} h_k^{(l)}]))} $

- $ h_i^{(l)} $: Características del nodo $ i $ en la capa $ l $.
- $ \mathcal{N}(i) $: Conjunto de nodos vecinos de $ i $.
- $ W^{(l)} $: Pesos de la capa $ l $.
- $ \alpha_{ij}^{(l)} $: Coeficientes de atención.
- $ a $: Pesos de la atención.
- $ \sigma $: Función de activación.
- $ \| $: Concatenación de vectores.

También puede funcionar bien con grafos heterofílicos, ya que utiliza mecanismos de atención para ponderar la importancia de los vecinos de cada nodo, lo que le permite capturar relaciones más complejas en la red.

## Resultados

Los resultados obtenidos se analizan con diferentes métricas. Estas son las siguientes:

* Accuracy:

  La accuracy es la proporción de predicciones correctas (tanto verdaderos positivos como verdaderos negativos) entre todas las predicciones realizadas. Es una métrica simple y directa que muestra el porcentaje de predicciones correctas. Sin embargo, en casos de clases desbalanceadas, puede ser engañosa, ya que un modelo puede tener una alta accuracy simplemente prediciendo siempre la clase mayoritaria.

* Precission:

  La precision es la proporción de verdaderos positivos entre todas las instancias predichas como positivas. Es crucial cuando el costo de los falsos positivos es alto.

* Recall:

  El recall es la proporción de verdaderos positivos entre todas las instancias que realmente son positivas. Es esencial cuando es crítico identificar todas las instancias positivas.

* F1 Score:

  El F1 score es la media armónica de la precision y el recall. Equilibra las dos métricas y proporciona una medida única del rendimiento del modelo. Es útil cuando se necesita un equilibrio entre precision y recall, especialmente en situaciones donde hay una desbalance en las clases.


Cada una de estas métricas ofrece una perspectiva diferente del rendimiento del modelo, y su importancia varía según el contexto y los objetivos específicos del problema de clasificación.

En la siguiente tabla se recogen los resultados obtenidos para cada dataset con los 5 modelos propuestos.


|  | Cora | Citeseer | Actor | Chamaleon | Wisconsin | Squirrel | Cornell | Texas |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| MLP | 75.074 $\pm{1.562}$ | 71.631 $\pm{1.703}$ | 35.800 $\pm{0.961}$ | 50.000 $\pm{1.609}$ | 90.588 $\pm{5.601}$ | 36.676 $\pm{1.906}$ | 87.297 $\pm{5.927}$ | 92.432 $\pm{3.585}$ |
| GCN | 80.590 $\pm{1.062}$ | 72.852 $\pm{1.000}$ | 30.658 $\pm{1.326}$ | 33.136 $\pm{4.054}$ | 70.392 $\pm{5.976}$ | 27.810 $\pm{2.304}$ | 67.027 $\pm{7.232}$ | 71.108 $\pm{6.598}$ |
| SAGE | 80.775 $\pm{2.446}$ | 73.123 $\pm{1.423}$ | 34.526 $\pm{1.029}$ | 45.000 $\pm{1.028}$ | 89.804 $\pm{5.462}$ | 35.600 $\pm{1.598}$ | 82.703 $\pm{6.068}$ | 89.459 $\pm{5.331}$ |
| Cheb (K=10/K=2) | 81.790 $\pm{1.883}$ | 72.312 $\pm{1.577}$ | 35.717 $\pm{0.952}$ | 49.627 $\pm{1.693}$ | 90.196 $\pm{4.960}$ | 37.454 $\pm{1.699}$ | 83.784 $\pm{7.451}$ | 90.270 $\pm{5.695}$ |
| GAT | 79.981 $\pm{1.506}$ | 72.492 $\pm{1.379}$ | 32.246 $\pm{0.881}$ | 28.202 $\pm{6.085}$ | 76.275 $\pm{5.912}$ | 26.916 $\pm{1.881}$ | 79.189 $\pm{5.927}$ | 79.459 $\pm{5.822}$ |


Si nos fijamos en el modelo MLP, este funciona mejor para grafos heterofílicos. Resulta ser el mejor modelo o el segundo mejor para este tipo de datasets. En cambio, para los datasets de Cora y Citeseer que son grafos homofílicos funcionan mejor el resto de modelos.

Para la GCN vemos que es mejor aplicarla en grafos homofílicos que en heterofílicos, pues al tener en cuenta el vecindario del nodo a la hora de clasificar se ve más contaminado. Añadir más capas no mejora los resultados obtenidos.

SAGE agrega información de los vecinos con unos pesos y mediante una función de agregación, lo que mejora las predicciones. Cabe destacar que para grafos homofílicos se ha probado un modelo de dos capas convolucionales cuya primera capa tiene como hiperparámetro ``aggr='max'`` y la segunda ``aggr='mean'`` que es la que viene por defecto. Esto mejora sus predicciones ya que la mayoría son de un mismo tipo en un grafo homofílico. Sin embargo para heterofílicos ambas capas llevan ``aggr='mean'`` que obtiene un mejor resultado.

Con el modelo de Cheb se han realizado pruebas con ``K=2`` y ``K=10``, para homofílicos es mejor aumentar K, en cambio para heterofílicos los resultados empeoran bastante, por ello nos quedamos con ``K=2`` en estos casos. Para Actor empeora la predicción a un 31.9%, al igual que para Chamaleon, para Wisconsin con K=10 baja a un 72.9%, para Cornell baja a un 66.5% y para Texas baja a 74.3%.

Con el modelo GAT las predicciones no son destacables, en grafos homofílicos superan la MLP pero están por debajo del resto de GCNs y en grafos heterofílicos es mejor que la GCN simple pero está por debajo del resto. En el caso de Chamaleon es el peor modelo de los propuestos.

Para los datasets de Actor, Chamaleon y Squirrel ninguno de los modelos consigue unas predicciones buenas, en cambio para los otros heterofílicos, Texas, Wisconsin y Cornell la MLP consigue resultados que rondan el 90%. El tamaño del grafo puede influir en el rendimiento. Grafos más grandes como Chameleon y Squirrel pueden ser más desafiantes para los modelos debido a la mayor complejidad y el mayor número de nodos y aristas. Los grafos más pequeños como Texas, Wisconsin y Cornell pueden ser más manejables y permitir que los modelos capten patrones más fácilmente. En la representación del grafo de Chamaleon se observa como está organizado en comunidades lo que puede hacer que obtenga mejores resultados que Squirrel y Actor.